---
title: Personalized prediction correlations for rn7 candidate genes
date: 9/1/23
author: Sabrina Mi
---

We ran the personalized Enformer pipeline 868 genes for high variation in Br rats and number of eQTLs.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [5]:
with open("gene_list.txt", "r") as file:
    gene_list = file.read().splitlines()

In [18]:
predictions_dir = "/home/s1mi/Br_predictions/predictions_folder/personalized_Br_selected_genes/predictions_2023-09-01/enformer_predictions"
gene_expr_bed = "/home/s1mi/enformer_rat_data/expression_data/Brain.rn7.expr.tpm.bed"
obs_gene_expr = pd.read_csv(gene_expr_bed, sep="\t", header=0, index_col='gene_id')
annot_df = pd.read_csv("/home/s1mi/enformer_rat_data/annotation/rn7.gene.txt", sep="\t", header= 0, index_col='geneId')

/tmp/ipykernel_144779/3167660153.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  obs_gene_expr = pd.read_csv(gene_expr_bed, sep="\t", header=0, index_col='gene_id')
/tmp/ipykernel_144779/3167660153.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  annot_df = pd.read_csv("/home/s1mi/enformer_rat_data/annotation/rn7.gene.txt", sep="\t", header= 0, index_col='geneId')


In [11]:
expr_dict = {}
for gene in gene_list:
    obs = obs_gene_expr.loc[gene][3:]
    expr_dict[gene] = pd.DataFrame({"observed": obs})

In [ ]:
for gene in gene_list:
    gene_annot = annot_df.loc[gene]
    interval = f"chr{gene_annot['chromosome']}_{gene_annot['tss']}_{gene_annot['tss']}"
    human_predicted = []
    mouse_predicted = []
    for individual in expr_dict[gene].index:
        haplo0 = h5py.File(f"{predictions_dir}/{individual}/haplotype0/{interval}_predictions.h5", "r")
        human_prediction = haplo0["human"][446:450, 4980]
        mouse_prediction = haplo0["mouse"][446:450, 1300]
        human_predicted.append(np.average(human_prediction))
        mouse_predicted.append(np.average(mouse_prediction))
    expr_dict[gene]["human predicted"] = human_predicted
    expr_dict[gene]["mouse predicted"] = mouse_predicted